In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek2 import env
from ronek2 import const
from ronek2 import backend as bkd
from ronek2.systems import BoxAd

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
model = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True
)
model.set_output(max_mom=2, linear=True)

In [6]:
sol_ref = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/argon/testcase/test_ad/output_box"

In [ ]:
columns = "t X_em X_Ar X_Arp Th Te rho p n E".split()
box = np.loadtxt(sol_ref + "/box.dat")
box = pd.DataFrame(box, columns=columns)
box

In [8]:
t = box["t"].values
rho, Th0, Te0 = [box[k][0] for k in ("rho", "Th", "Te")]

In [9]:
def read_pop(name):
  ni_gi  = np.loadtxt(sol_ref + f"/pop_{name}.dat", comments="&", skiprows=2)[:,1]
  levels = model.mix.species[name].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [10]:
n = {k: read_pop(k) for k in ("Ar", "Arp")}
n["em"] = (box["X_em"] * box["n"]).values.reshape(-1,1)
n_true = np.hstack([n[k] for k in model.species_order]).T
rho_true = model.mix.get_rho(bkd.to_torch(n_true)).numpy()
T_true = box[["Th", "Te"]].values.T
rho = np.sum(rho_true[:,0])

In [ ]:
y0 = np.zeros(model.nb_eqs)
y0[:model.mix.nb_comp] = rho_true[:,0] / rho
y0[-2] = Th0
y0[-1] = model.mix.get_pe(Te=Te0, ne=n_true[-1,0])
y0

In [ ]:
y, runtime = model.solve_fom(t, y0, rho)
runtime

In [13]:
w_pred = copy.deepcopy(y[:model.mix.nb_comp])
n_pred = model.mix.get_n(bkd.to_torch(w_pred)).numpy()
T_pred = copy.deepcopy(y[model.mix.nb_comp:])
T_pred[-1] = model.mix.get_Te(pe=T_pred[-1], ne=n_pred[-1])

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.semilogx(t[1:], T_true[i][1:])
  plt.semilogx(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_true[i] - n_pred[i])/n_true[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

Linear model

In [16]:
def solve_lin(t, y0):
  A = model.jac(0.0, y0)
  b = model.fun(0.0, y0)
  y = sp.integrate.solve_ivp(
    fun=lambda t, y: A @ y + b,
    t_span=[0.0,t[-1]],
    y0=np.zeros_like(y0),
    method="LSODA",
    t_eval=t,
    first_step=1e-14,
    rtol=1e-6,
    atol=1e-20,
    jac=lambda t, y: A
  ).y
  y += y0.reshape(-1,1)
  return y

In [17]:
ylin = solve_lin(t, y0)

In [18]:
w_lin = copy.deepcopy(ylin[:model.mix.nb_comp])
n_lin = model.mix.get_n(bkd.to_torch(w_lin)).numpy()
T_lin = copy.deepcopy(ylin[model.mix.nb_comp:])
T_lin[-1] = model.mix.get_Te(pe=T_lin[-1], ne=n_lin[-1])

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_pred[i] - T_lin[i])/T_pred[i]))
  err.append(ierr)
  plt.semilogx(t[1:], T_pred[i][1:])
  plt.semilogx(t[1:], T_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_pred[i] - n_lin[i])/n_pred[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_pred[i][1:])
    plt.loglog(t[1:], n_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()

Sample time points

In [21]:
y_interp = sp.interpolate.interp1d(t, y, kind="linear", axis=-1)

In [22]:
tvec = np.array([0.0] + np.geomspace(1e-12, 1e-4, 100).tolist())
tsamples = np.geomspace(1e-13, 1e-4, 10)

In [ ]:
for i, ti in enumerate(tsamples):
  
  print(f"Time {i+1}: {ti} s")
  print("="*40)

  yref = y_interp(tvec+ti)
  ylin = solve_lin(tvec, yref[:,0])

  for s in model.mix.species.values():
    for i in s.indices:
      plt.loglog(tvec[1:], yref[i][1:])
      plt.loglog(tvec[1:], ylin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
    plt.show()
    plt.close()